In [12]:
import os
import pickle as pkl

from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

SAVE = False  # whether to save the outputs of this notebook

if SAVE:
    # Replace this with the path to directory where you would like results to be saved
    OUTPUT_PATH = "../output_files/synthetic_example"
    os.makedirs(OUTPUT_PATH, exist_ok=True)  # Create path if it doesn't exist

In [2]:
counts_df = load_example_data(
    modality="raw_counts",
    dataset="synthetic",
    debug=False,
)

clinical_df = load_example_data(
    modality="clinical",
    dataset="synthetic",
    debug=False,
)

print(counts_df)

# %%
print(clinical_df)

           gene1  gene2  gene3  gene4  gene5  gene6  gene7  gene8  gene9  \
sample1       12     21      4    130     18      0     16     54     49   
sample2        1     44      2     63     11     10     70     32     57   
sample3        4      4     11    180     21      3     28     34     65   
sample4        1     10      2    100     44      9     28     16     33   
sample5        1     11      6    135     16      2     32     29     31   
...          ...    ...    ...    ...    ...    ...    ...    ...    ...   
sample96       7     26      3     67     11      4     41     44     54   
sample97       1     14      3     71     33      5     19     42     25   
sample98      10     36      2     72     11      2     66     27     16   
sample99      18     14      3     66     53     11     32     19     79   
sample100     21      9      3     42     13     13     19     78     30   

           gene10  
sample1         3  
sample2         9  
sample3         2  
sample4

In [3]:
samples_to_keep = ~clinical_df.condition.isna()
counts_df = counts_df.loc[samples_to_keep]
clinical_df = clinical_df.loc[samples_to_keep]


In [4]:
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]

In [5]:
dds = DeseqDataSet(
    counts=counts_df,
    clinical=clinical_df,
    design_factors="condition",
    refit_cooks=True,
    n_cpus=8,
)

In [6]:
dds.deseq2()


if SAVE:
    with open(os.path.join(OUTPUT_PATH, "dds.pkl"), "wb") as f:
        pkl.dump(dds, f)

# %%
# The :class:`DeseqDataSet` class extends the
# :class:`AnnData <anndata.AnnData>`
# class.

print(dds)

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.03 seconds.

Fitting dispersion trend curve...
... done in 0.04 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.02 seconds.

Refitting 0 outliers.

AnnData object with n_obs × n_vars = 100 × 10
    obs: 'condition', 'group'
    uns: 'trend_coeffs', '_squared_logres', 'prior_disp_var'
    obsm: 'design_matrix', 'size_factors', 'replaceable'
    varm: 'non_zero', '_rough_dispersions', 'genewise_dispersions', '_genewise_converged', '_normed_means', 'fitted_dispersions', 'MAP_dispersions', '_MAP_converged', 'dispersions', '_outlier_genes', 'LFC', '_LFC_converged', 'replaced'
    layers: 'normed_counts', '_mu_hat', '_mu_LFC', '_hat_diagonals', 'cooks'


In [7]:
print(dds.varm["dispersions"])

# %%

print(dds.varm["LFC"])

[0.88259868 0.22257863 0.83723801 0.15897048 0.24992589 0.97364795
 0.23515487 0.19878078 0.1865203  0.63189989]
        intercept  condition_B_vs_A
gene1    1.891436          0.438632
gene2    2.851662          0.373296
gene3    1.787780         -0.438645
gene4    4.741958         -0.285647
gene5    3.077798          0.403457
gene6    1.678536          0.001010
gene7    3.291025          0.093116
gene8    3.785129         -0.187604
gene9    3.682882         -0.147443
gene10   2.300515          0.267562


In [8]:
stat_res = DeseqStats(dds, n_cpus=8)

In [9]:
stat_res.summary()

if SAVE:
    with open(os.path.join(OUTPUT_PATH, "stat_results.pkl"), "wb") as f:
        pkl.dump(stat_res, f)

Running Wald tests...
... done in 0.01 seconds.

Log2 fold change & Wald test p-value: condition B vs A


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene1,8.541317,0.632812,0.289101,2.188898,0.028604,0.064150
gene2,21.281239,0.538552,0.149963,3.591235,0.000329,0.001646
gene3,5.010123,-0.632830,0.295236,-2.143475,0.032075,0.064150
gene4,100.517961,-0.412102,0.118629,-3.473867,0.000513,0.001710
gene5,27.142450,0.582065,0.154706,3.762408,0.000168,0.001646
gene6,5.413043,0.001457,0.310311,0.004696,0.996253,0.996253
gene7,28.294023,0.134338,0.149945,0.895917,0.370297,0.411441
gene8,40.358344,-0.270656,0.136401,-1.984260,0.047227,0.078711
gene9,37.166183,-0.212715,0.133243,-1.596437,0.110391,0.143148
gene10,11.589325,0.386011,0.244588,1.578207,0.114518,0.143148


In [10]:
if SAVE:
    with open(os.path.join(OUTPUT_PATH, "shrunk_stat_results.pkl"), "wb") as f:
        pkl.dump(stat_res, f)

In [11]:
print(stat_res.shrunk_LFCs)  # Will be True only if lfc_shrink() was run.

False


In [20]:
print(summary(dds))
# padj.cutoff <- 0.05
# stat_res <- ________ %>% filter(padj < padj.cutoff)

NameError: name 'summary' is not defined